<a href="https://colab.research.google.com/github/Naresh0298/100DaysChallege/blob/main/CNN_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader



In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("samithsachidanandan/human-face-emotions")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'human-face-emotions' dataset.
Path to dataset files: /kaggle/input/human-face-emotions


In [3]:
data  = path + '/Data'

In [4]:
data

'/kaggle/input/human-face-emotions/Data'

In [5]:
import os

# List the contents of the 'Data' directory
print(os.listdir(data))

['Fear', 'Suprise', 'Angry', 'Sad', 'Happy']


In [6]:
from torchvision.datasets import ImageFolder
raw_dataset = datasets.ImageFolder(root=data)
classes = raw_dataset.classes

In [7]:
train

NameError: name 'train' is not defined

In [ ]:
raw_dataset.class_to_idx

In [ ]:
train_trans = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),

    transforms.Normalize([0.485,0.456,0.406],
                         [0.229,0.224,0.225])

])

test_trans = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

In [ ]:
from torch.utils.data import random_split
train_subset, val_subset, test_subset = random_split(
    raw_dataset,
    [0.7,0.2,0.1],
    generator = torch.manual_seed(2)
)

In [ ]:
class TransformedSubset(torch.utils.data.Dataset):
    def __init__(self, subset, transform=None):
        self.subset = subset
        self.transform = transform

    def __getitem__(self, index):
        x, y = self.subset[index]
        if self.transform:
            x = self.transform(x)
        return x, y

    def __len__(self):
        return len(self.subset)

train_data = TransformedSubset(train_subset, transform=train_trans)
val_data = TransformedSubset(val_subset, transform=test_trans)
test_data = TransformedSubset(test_subset, transform=test_trans)

In [ ]:
raw_dataset.classes

In [ ]:
val_loader = DataLoader(val_data, shuffle = True, batch_size = 64)

In [ ]:
train_loader = DataLoader(train_data, shuffle = True, batch_size = 64)

In [ ]:
print(f"Number of samples in training dataset: {len(train_data)}")
print(f"Number of samples in validation dataset: {len(val_data)}")
print(f"Number of samples in test dataset: {len(test_data)}")

In [ ]:
train_data[0][0].shape


In [ ]:
from torch.nn.modules import padding
from torch.nn.modules.pooling import MaxPool2d
class CNN(nn.Module):
  def __init__(self):
    super().__init__()
    self.feature = nn.Sequential(
        nn.Conv2d(3, 32, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2), # 32x16x16
        nn.Conv2d(32, 64, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2), # 64x8x8
        nn.Conv2d(64, 128, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2), # 128x4x4
      )

    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(128 * 4 * 4, 512), nn.ReLU(), nn.Dropout(0.5),
        nn.Linear(512, 5) # 5 classes
    )

  def forward(self,x):
    return self.classifier(self.feature(x))

In [ ]:
model = CNN()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=0.001)

In [ ]:
epochs = 20

for epoch in range(epochs):

  epoch_loss = 0.0;

  for inputs, labels in train_loader:
    inputs = inputs.to(device)
    labels = labels.to(device)

    optimizer.zero_grad()

    outputs = model(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    epoch_loss+=loss.item()

  ##val
  val_loss=0.0
  model.eval()
  for inputs, labels in val_loader:

    inputs = inputs.to(device)
    labels = labels.to(device)


    outputs = model(inputs)
    loss = criterion(outputs, labels)

    val_loss+=loss.item()


  print(f'Epoch :{epoch} , train Loss: {epoch_loss/len(train_loader)} , val Loss: {val_loss/len(val_loader)}')



In [ ]:
test_loader = DataLoader(test_data, shuffle=False, batch_size=64)

In [ ]:
model.eval()  # Set the model to evaluation mode
correct = 0
total = 0

with torch.no_grad():  # Disable gradient calculation during evaluation
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the model on the test images: {100 * correct / total:.2f}%')

TypeError: 'module' object is not callable

In [87]:
test_data[0][0]

tensor([[[-0.6794,  1.8722,  2.0092,  ...,  0.3309,  0.3652, -1.1760],
         [-0.6794,  1.8208,  2.0092,  ...,  0.2111,  0.2453, -1.2274],
         [-0.7308,  1.7865,  1.9920,  ...,  0.1083,  0.1083, -1.2788],
         ...,
         [-1.7240, -0.8507, -0.7822,  ..., -0.6109, -0.6109, -1.5528],
         [-1.7754, -0.8507, -0.6109,  ..., -0.5424, -0.5767, -1.5357],
         [-1.7240, -0.8849, -0.5253,  ..., -0.4397, -0.4568, -1.4843]],

        [[-0.5651,  2.0434,  2.1835,  ...,  0.4678,  0.5028, -1.0728],
         [-0.5651,  1.9909,  2.1835,  ...,  0.3452,  0.3803, -1.1253],
         [-0.6176,  1.9559,  2.1660,  ...,  0.2402,  0.2402, -1.1779],
         ...,
         [-1.6331, -0.7402, -0.6702,  ..., -0.4951, -0.4951, -1.4580],
         [-1.6856, -0.7402, -0.4951,  ..., -0.4251, -0.4601, -1.4405],
         [-1.6331, -0.7752, -0.4076,  ..., -0.3200, -0.3375, -1.3880]],

        [[-0.3404,  2.2566,  2.3960,  ...,  0.6879,  0.7228, -0.8458],
         [-0.3404,  2.2043,  2.3960,  ...,  0